In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src/'))

### Data preparationdataset

In [ ]:
import cicids2017
from pathlib import Path
from ProphetStorm import ProphetStorm


RES_PATH = "."
DATASETPATH_CACHE = Path(f"../dataset/cache")
DATASETPATH = Path("../dataset/CICIDS2017_ntop.pkl")


dataset = cicids2017.load_dataset(DATASETPATH_CACHE)
ps = ProphetStorm()
ps.fit(dataset["TR"])

Importing plotly failed. Interactive plots will not work.
 71%|███████▏  | 10/14 [41:32<11:31, 172.97s/it] 

### Training prophet storm

In [ ]:
import pandas as pd

dt = dataset["DT"]

netdata = ps.isanomaly(dt)
Y = pd.concat(dt)["_isanomaly"]
Y = (Y=="none").replace({True: 0, False: 1}).values
y_hat = netdata["_y_hat"].values

In [6]:
from sklearn import metrics


metrics_rep = [ metrics.roc_auc_score,
                metrics.precision_score, metrics.recall_score,
                metrics.accuracy_score, metrics.f1_score]
for m in metrics_rep:
    mres = m(Y, y_hat)
    print(f"{m.__name__}(moday+attacks): {mres}")

tn, fp, fn, tp = metrics.confusion_matrix(Y, y_hat, normalize="all").ravel()
print("\n Confusion matrix")
print(f"\ttp: {tp} \tfp: {fp} \n\tfn: {fn} \ttn: {tn}")

roc_auc_score(moday+attacks): 0.5
precision_score(moday+attacks): 0.15090460526315788
recall_score(moday+attacks): 1.0
accuracy_score(moday+attacks): 0.15090460526315788
f1_score(moday+attacks): 0.26223651304037154

 Confusion matrix
	tp: 0.15090460526315788 	fp: 0.8490953947368421 
	fn: 0.0 	tn: 0.0


In [5]:
!LS

CICIDS2017_GridSearch.ipynb        Kitsune.ipynb
CICIDS2017_ScoreSigma.ipynb        Prophet.ipynb
CICIDS2017_T-SNE_Visualizer.ipynb  RenameMe.png
CICIDS2017_VisualizationTool.ipynb TripletLoss.ipynb
